In [5]:
import tensorflow as tf
import numpy as np
import os

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, AveragePooling1D, Conv1D, DenseFeatures, Input, Flatten, GlobalAveragePooling1D

# Loading Dataset

Our dataset contains four directories. Each directory referst to one class of motion to be classified. Each directory contains 395 recordings. 

---
## Recordings
Each line of the recording is the average value of the gyro sensor over 10 ms. The whole recording time is 2000ms. Therefore each recording has 200 lines.
Each line of the recording contais six different values. They are the acceleration values on all three spatial axes as well as the gyroscope information on all three spatial axes. 

> The accelerometer values are deliberately not cleaned from earths acceleration.
---
## Reading the data

For the reading of the data we iterate over all directories in the dataset directory, and the over each file in the gesture directory. We then save the data into a python list. We also save the desired classification outcome, at the same time, into another list. That way you will find the classification of the dataset at the same index of the label list as the data in the data list.


In [6]:
dataset_dir = "gestures"

gestures = ["Computer", "Elephant", "Sorry", "No-Sign"]

dataset_values = []
dataset_labels = []

for gesture in gestures:
    for filename in os.listdir(f"{dataset_dir}/{gesture}"):
        with open(os.path.join(f"{dataset_dir}/{gesture}", filename), 'r') as file:
            file_data = []

            text = file.read()
            lines = text.splitlines()

            for line in lines:
                int_values = []
                for value in line.split():
                    int_values.append(int(value))

                file_data.append(int_values)

            dataset_values.append(file_data)

            if gesture == "Computer":
                dataset_labels.append([1, 0, 0, 0])
            elif gesture == "Elephant":
                dataset_labels.append([0, 1, 0, 0])
            elif gesture == "Sorry":
                dataset_labels.append([0, 0, 1, 0])
            elif gesture == "No-Sign":
                dataset_labels.append([0, 0, 0, 1])
            else:
                print("You did not change the labeling of the dataset you blithering idiot")

dataset_values = np.array(dataset_values)
dataset_labels = np.array(dataset_labels)

print(dataset_values.shape)
print(dataset_labels.shape)

(1584, 200, 6)
(1584, 4)


## Creating the dataset

We convert the two lists mentioned above into a tf.Dataset. 

---

First we define SHUFFLE_BUFFER_SIZE and VALIDATION_SIZE. We want to shuffle the complete Dataset, therefore the shuffle buffer has the same size as the dataset itself. We want to have Validationset that is the size of about 30% of the whole Dataset.

We create our dataset and shuffle it.

For the validation set we take the defined amount from the front of the shuffled dataset. The rest is our training data.

In [13]:
DATASET_SIZE = len(dataset_values)

SHUFFLE_BUFFER_SIZE = DATASET_SIZE
VALIDATION_SIZE = int(DATASET_SIZE * 0.3)

BATCH_SIZE = 64

dataset = tf.data.Dataset.from_tensor_slices((dataset_values, dataset_labels))
dataset = dataset.shuffle(SHUFFLE_BUFFER_SIZE)

validation_dataset = dataset.take(VALIDATION_SIZE).batch(BATCH_SIZE)  
train_dataset = dataset.skip(VALIDATION_SIZE).batch(BATCH_SIZE)

# for values, labels in train_dataset.take(1):
    # print(labels.numpy(), values.numpy())

475


## The Model

---

In [17]:
%%time
model = Sequential([
    Conv1D(filters = 125, strides = 3, kernel_size=70, activation='relu', input_shape=(200, 6)),
    GlobalAveragePooling1D(),
    Dense(100, activation='relu'),
    Dense(20, activation='relu'),
    Dense(4, activation='softmax')
])

model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])
model.build(input_shape=(1584, 200, 6))
model.summary()
model.fit(train_dataset, validation_data=validation_dataset, epochs=25)

model.save("./model", save_format='tf')
model.save("./model.h5", save_format='h5')


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_4 (Conv1D)           (None, 44, 125)           52625     
                                                                 
 global_average_pooling1d_4   (None, 125)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_12 (Dense)            (None, 100)               12600     
                                                                 
 dense_13 (Dense)            (None, 20)                2020      
                                                                 
 dense_14 (Dense)            (None, 4)                 84        
                                                                 
Total params: 67,329
Trainable params: 67,329
Non-trainable params: 0
__________________________________________________

2022-04-18 13:08:08.240769: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


18/18 [==============================] - 1s 19ms/step - loss: 288.7202 - accuracy: 0.7403 - val_loss: 62.8104 - val_accuracy: 0.9347
Epoch 2/25
 7/18 [==========>...................] - ETA: 0s - loss: 55.1617 - accuracy: 0.9397 

2022-04-18 13:08:08.577351: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


18/18 [==============================] - 0s 12ms/step - loss: 52.9018 - accuracy: 0.9279 - val_loss: 16.5763 - val_accuracy: 0.9389
Epoch 3/25
18/18 [==============================] - 0s 12ms/step - loss: 44.2443 - accuracy: 0.9170 - val_loss: 37.3393 - val_accuracy: 0.9326
Epoch 4/25
18/18 [==============================] - 0s 12ms/step - loss: 34.0358 - accuracy: 0.9459 - val_loss: 24.9678 - val_accuracy: 0.9684
Epoch 5/25
18/18 [==============================] - 0s 12ms/step - loss: 29.6947 - accuracy: 0.9477 - val_loss: 12.8926 - val_accuracy: 0.9600
Epoch 6/25
18/18 [==============================] - 0s 11ms/step - loss: 23.4718 - accuracy: 0.9513 - val_loss: 11.7683 - val_accuracy: 0.9705
Epoch 7/25
18/18 [==============================] - 0s 12ms/step - loss: 12.6761 - accuracy: 0.9594 - val_loss: 11.3155 - val_accuracy: 0.9726
Epoch 8/25
18/18 [==============================] - 0s 12ms/step - loss: 10.5475 - accuracy: 0.9630 - val_loss: 12.8120 - val_accuracy: 0.9768
Epoch 9/25

INFO:tensorflow:Assets written to: ./model/assets


CPU times: user 5.2 s, sys: 2.91 s, total: 8.11 s
Wall time: 5.9 s


In [9]:
modeldir = 'models/0.9900/model'
converter = tf.lite.TFLiteConverter.from_saved_model(modeldir)
tflite_model = converter.convert()

open("model.tflite", "wb").write(tflite_model)

2022-04-18 12:59:10.206566: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2022-04-18 12:59:10.206577: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2022-04-18 12:59:10.206650: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: models/0.9900/model
2022-04-18 12:59:10.207464: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2022-04-18 12:59:10.207469: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: models/0.9900/model
2022-04-18 12:59:10.209550: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-04-18 12:59:10.234875: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: models/0.9900/model
2022-04-18 12:59:10.242003: I tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serve }; Status: success: OK. Too

324836